In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# INTRODUCTION: 

1. ANALYSIS OF DATA

# IMPORTING ALL NECESSARY LIBRARIES FOR ANALYSIS



In [ ]:
import numpy as np 
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import warnings
warnings.simplefilter("ignore")

# READING DATA USING PANDAS LIBRARY

In [ ]:
data_df = pd.read_csv("/kaggle/input/reddit-vaccine-myths/reddit_vm.csv")

# VIEWING ONLY TOP 5 VALUES TO CHECK THE FILE

In [ ]:
data_df.head()

# CHECKING ANY MISSING DATA

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
missing_data(data_df)

# OUR DATA IS MISSNG HALF OF THE POSTS

# NOW VISUALIZING THE DATA

1. WORDCLOUDS ARE AMAZING TO REPRESENT THE MOST FREQUENT WORDS IN TITLES AND BODY OF POST
2. UNNCESSARY FREQUENT WORDS ARE REMOVED LIKE, 'VACCINE', 'VACCINATION',  AND 'COMMENTS'.   KEYWORD COMMENTS WILL BE USED FOR TITLE.




In [ ]:
def show_wordcloud(data, title=""):
    text = " ".join(t for t in data.dropna())
    stopwords = set(STOPWORDS)
    stopwords.update(["t", "co", "https", "amp", "U", "fuck", "fucking", "Comment", "vaccination", "vaccine", "vaccines", "vaccinate", "vaccinated"])
    wordcloud = WordCloud(stopwords=stopwords, scale=4, max_font_size=50, max_words=500,background_color="black").generate(text)
    fig = plt.figure(1, figsize=(16,16))
    plt.axis('off')
    fig.suptitle(title, fontsize=20)
    fig.subplots_adjust(top=2.3)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.show()

# WORDCLOUD  SHOWING TITLES

In [ ]:
show_wordcloud(data_df['title'], title = 'Prevalent words in titles')

# WORDCLOUD REPRESENTTING BODY 

In [ ]:
show_wordcloud(data_df['body'], title = 'Prevalent words in post bodies')

# SENTIMENTAL ANALYSIS

1. USING NATURAL LANGUAGE TOOLKIT 

SentimentIntensityAnalyzer()

In [ ]:
# borrowed from https://www.kaggle.com/pashupatigupta/sentiments-transformer-vader-embedding-bert
sia = SentimentIntensityAnalyzer()
def find_sentiment(post):
    if sia.polarity_scores(post)["compound"] > 0:
        return "Positive"
    elif sia.polarity_scores(post)["compound"] < 0:
        return "Negative"
    else:
        return "Neutral"       

In [ ]:
def plot_sentiment(df, feature, title):
    counts = df[feature].value_counts()
    percent = counts/sum(counts)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

    counts.plot(kind='bar', ax=ax1, color='green')
    percent.plot(kind='bar', ax=ax2, color='blue')
    ax1.set_ylabel(f'Counts : {title} sentiments', size=12)
    ax2.set_ylabel(f'Percentage : {title} sentiments', size=12)
    plt.suptitle(f"Sentiment analysis: {title}")
    plt.tight_layout()
    plt.show()

# PLOTTING TITLES USING LAMBDA FUNCTIONS

In [ ]:
data_df['title_sentiment'] = data_df['title'].apply(lambda x: find_sentiment(x))
plot_sentiment(data_df, 'title_sentiment', 'Title')

In [ ]:
show_wordcloud(data_df.loc[data_df['title_sentiment']=='Positive', 'title'], title = 'Prevalent words in titles (Positive sentiment)')

In [ ]:
show_wordcloud(data_df.loc[data_df['title_sentiment']=='Negative', 'title'], title = 'Prevalent words in titles (Negative sentiment)')

In [ ]:
show_wordcloud(data_df.loc[data_df['title_sentiment']=='Neutral', 'title'], title = 'Prevalent words in titles (Neutral sentiment)')

# REPRESENTING BOBY

In [ ]:
df = data_df.loc[~data_df.body.isna()]
df['body_sentiment'] = df['body'].apply(lambda x: find_sentiment(x))
plot_sentiment(df, 'body_sentiment', 'Body')

In [ ]:
show_wordcloud(df.loc[df['body_sentiment']=='Positive', 'body'], title = 'Prevalent words in body (Positive sentiment)')

In [ ]:
show_wordcloud(df.loc[df['body_sentiment']=='Negative', 'body'], title = 'Prevalent words in body (Negative sentiment)')

In [ ]:
show_wordcloud(df.loc[df['body_sentiment']=='Neutral', 'body'], title = 'Prevalent words in body (Neutral sentiment)')

# USING TEXTBLOB LIBRARY

1. TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more

In [ ]:
def find_sentiment_polarity_textblob(post):
    blob = TextBlob(post)
    polarity = 0
    for sentence in blob.sentences:
        polarity += sentence.sentiment.polarity
    return polarity

def find_sentiment_subjectivity_textblob(post):
    blob = TextBlob(post)
    subjectivity = 0
    for sentence in blob.sentences:
        subjectivity += sentence.sentiment.subjectivity
    return subjectivity

In [ ]:
data_df['title_sentiment_polarity'] = data_df['title'].apply(lambda x: find_sentiment_polarity_textblob(x))
data_df['title_sentiment_subjectivity'] = data_df['title'].apply(lambda x: find_sentiment_subjectivity_textblob(x))

In [ ]:
def plot_sentiment_textblob(df, feature, title):
    polarity = df[feature+'_sentiment_polarity']
    subjectivity = df[feature+'_sentiment_subjectivity']

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

    polarity.plot(kind='kde', ax=ax1, color='magenta')
    subjectivity.plot(kind='kde', ax=ax2, color='green')
    ax1.set_ylabel(f'Sentiment polarity : {title}', size=12)
    ax2.set_ylabel(f'Sentiment subjectivity: {title}', size=12)
    plt.suptitle(f"Sentiment analysis (polarity & subjectivity): {title}")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_sentiment_textblob(data_df, "title", 'Title')

In [ ]:
df['body_sentiment_polarity'] = df['body'].apply(lambda x: find_sentiment_polarity_textblob(x))
df['body_sentiment_subjectivity'] = df['body'].apply(lambda x: find_sentiment_subjectivity_textblob(x))

In [ ]:
plot_sentiment_textblob(df, "body", 'Body')